Scratchpad for loading data into the data

In [1]:
%reload_ext autoreload
%autoreload 2

import glob
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker
import json as js
from mpp.models import BlendedMetadata

In [2]:
with open('../local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

files = glob.glob('outputs/osdds/*.json')
    
# our connection
engine = sqla.create_engine(conf.get('connection'))
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [ ]:
session.rollback()

In [7]:
# the fgdc with iso
# it's a list of lists, no reason

with open('outputs/fgdc_with_iso.txt', 'r') as f:
    data = js.loads(f.read())
    
for i, d in enumerate(data):
    rows = []
    for row in d:
        rows.append(
            BlendedMetadata(
                response_id=row.get('id'),
                tags=row.get('tags'),
                extracted_info=row.get('xml'),
                extracted_type='xpath:elements',
                source_standard='FGDC',
                unexpected_standard='ISO'
            )
        )
    
    session.add_all(rows)
    try:
        session.commit()
    except:
        print 'commit failed line: ', i
        session.rollback()

In [5]:
# the iso with fgdc comments

with open('outputs/iso_with_comments.txt', 'r') as f:
    data = js.loads(f.read())
    
for i, d in enumerate(data):
    rows = []
    for row in d:
        rows.append(
            BlendedMetadata(
                response_id=row.get('id'),
                tags=row.get('tags'),
                extracted_info=row.get('comments'),
                extracted_type='xpath:element:comments',
                source_standard='ISO',
                unexpected_standard='FGDC'
            )
        )
    
    session.add_all(rows)
    try:
        session.commit()
    except:
        print 'commit failed line: ', i
        session.rollback()